In [1]:
clean_up = True # if True, remove all gams related files from working folder before starting
%run stdPackages.ipynb

The file _gams_py_gdb0.gdx is still active and was not deleted.
The file _gams_py_gdb1.gdx is still active and was not deleted.
The file _gams_py_gdb4.gdx is still active and was not deleted.
The file _gams_py_gdb5.gdx is still active and was not deleted.
The file _gams_py_gdb6.gdx is still active and was not deleted.


*Load specific modules used here:*

In [2]:
os.chdir(d['py'])
import mCGE
from gmsPython import nestingTree
from valueShares import nestedShares, nestedShares_noOutputs

# NCP 

*Load test data:*

In [3]:
t0 = 2019 # baseline year
name = 'TestModelData'
db = GpyDB(os.path.join(d['data'], name), name = 'NCPdb', ws = d['work'])
# AggDB.subsetDB(db, db('s_p').union(db('s_i')).union(db('s_HH')))
dbIO = db.copy() # create copy without adjustments made along the way
ws = db.ws # use this as the main workspace throughout

Initialize model:

In [4]:
M = mCGE.NCP_CGE('NCPcge', database = db)
# M.db['rDepr'] = M.steadyStateDepr() # use this to adjust depreciation rates to get steady state investments in baseline year

## 1. Add modules

The class has prespecified methods for adding modules. The modules for this specific CGE are: Production, investment, consumer, government, trade, and emissions.

### 1.1. Production

Nesting tree:

In [5]:
nest = nestingTree.AggTree(name = 'P', trees = {'P': nestingTree.Tree('P', tree = db('nestProduction').to_list(), f = 'CES')})
nest(namespace = {str(n)+'_input': n for n in db('n')})

Initial guess for share parameters:

In [6]:
v_p = nestedShares(nest, ws = ws)
db_p = v_p(dbIO)

Add to database:

In [7]:
db.aom(db_p('mu').xs(t0), name = 'mu')
db.aom(adj.rc_pd(db_p('vD'), nest.get('int')).rename('qD'), name = 'qD', priority = 'first') # specify intermediate goods levels

Use static user cost as initial guess for price on durables (if no other has been provided):

In [8]:
db.aom(db('pD_dur'), name = 'pD', priority = 'first') # pD_dur includes an estimate of the long-run user-cost of capital

#### Add module

The model class ```NCP_CGE``` contains the default specifications for the production modules:
* Class: ```DynamicNCES``` with adjustment costs.
* Extensions: Include emissions with related abatement costs.
* Use the lump sum tax in calibration of tax transfers in the baseline year.
* Initialize as a "partial" module: This allows us to first solve the model without general equilibrium conditions and then subsequently with (with a solution close to the "true").

Thus, if we only add the nesting tree, the other options are build in:

In [9]:
[nest.db.__setitem__(k, M.db[k]) for k in ('txE', 'dqCO2')];
M.stdProduction(nest);

### 1.2. Investment

Nesting tree

In [10]:
nest = nestingTree.AggTree(name = 'I', trees = {'I': nestingTree.Tree('I', tree = db('nestInvestment').to_list(), f = 'CES')}, ws = ws)
nest(namespace = {str(n)+'_input':n for n in db('n')});

Define share parameters:

In [11]:
v = nestedShares(nest, ws = ws)
db_i = v(dbIO) 

Use share parameters and values to get initial values for intermediates:

In [12]:
db.aom(db_i('mu').xs(t0), name = 'mu') 
db.aom(adj.rc_pd(db_i('vD'), nest.get('int')).rename('qD'), name = 'qD', priority = 'first') # specify intermediate goods levels

#### Add module

The production of investment goods follow a simple nested CES function as well. The model class here uses the following default options:
* Class: ```StaticNECS``` without emissions.
* Use the tax on outputs to calibrate total tax transfers in the baseline year.
* Initialize as a "partial" module: This allows us to first solve the model without general equilibrium conditions and then subsequently with (with a solution close to the "true").

Thus, if we only add the nesting tree, the other options are build in:

In [13]:
M.stdInvestment(nest);

### 1.3. Consumer module

Nesting tree:

In [14]:
nest = nestingTree.AggTree(name = 'HH', trees = {'HH': nestingTree.Tree('HH', tree = db('nestHH').to_list(), f = 'CES')}, ws = ws)
nest(namespace = {str(n)+'_input': n for n in db('n')});

Value shares:

In [15]:
v = nestedShares_noOutputs(nest, ws = ws)
db_vs = v(dbIO) # one-liner that returns solution database

Add to main database:

In [16]:
db.aom(db_vs('mu').xs(t0), name = 'mu') 
db.aom(adj.rc_pd(db_vs('vD'), nest.get('int').union(nest.get('output'))).rename('qD'), name = 'qD', priority = 'first') # specify intermediate goods levels

#### Add module

The consumer is modelled as a Ramsey consumer with endogenous labor supply (RamseyGHH). Furthermore, we assume that household assets are tied up in domestic firms. This is the "index fund" extension. The model class here uses the following default options:
* Class: ```RamseyGHHIdxFund```.
* It uses the lump-sum transfer to target total government transfers in the baseline year.
* It adds a $j$-term to the budget in the initial period to match baseline spendings.
* Initialize as a "partial" module: This allows us to first solve the model without general equilibrium conditions and then subsequently with (with a solution close to the "true").

We need to add nesting tree + indication of labor-to-consumption aggregate:

In [17]:
nest.db['t0'] = db['t0'] # this is used in the initialization phase - before merging the databases
M.stdHousehold(nest, db('L2C'))

### 1.4. Government module

Nesting tree:

In [18]:
nest = nestingTree.AggTree(name = 'G', trees = {'G': nestingTree.Tree('G', tree = db('nestG').to_list(), f = 'CES')}, ws = ws)
nest(namespace = {str(n)+'_input': n for n in db('n')});

Value shares:

In [19]:
v = nestedShares_noOutputs(nest, ws = ws)
db_vs = v(dbIO)

Add to main database:

In [20]:
db.aom(db_vs('mu').xs(t0), name = 'mu') 
db.aom(adj.rc_pd(db_vs('vD'), nest.get('int').union(nest.get('output'))).rename('qD'), name = 'qD', priority = 'first') # specify intermediate goods levels

#### Add module

The government sector is modelled with a nested CES function to determine demand from other sectors', and balances the budget using taxes on households. The model class here uses the following default options:
* Class: ```StaticNECS```.
* It uses the lump-sum tax to calibrate total tax transfers in the baseline year.
* It uses the lump-sum tax on households to ensure a balanced budget in all years.
* It uses intial level of assets to target total expenditures in the baseline year.
* Initialize as a "partial" module: This allows us to first solve the model without general equilibrium conditions and then subsequently with (with a solution close to the "true").

Simply specifying the nesting structure adopts the assumptions above as standard:

In [21]:
nest.db['t0'] = db['t0'] # this is used in the initialization phase - before merging the databases
M.stdGovernment(nest);

### 1.5. Trade module

The Armington trade module relies on two main specifications: Mapping from domestic to foreign types of goods (stored as ```dom2for``` in the database), and ```dExport``` that specifies the types of goods that the foreign sector demands. These symbols are already in the database, so we do not have to specify it here (but can be adjusted in case we need to treat trade with certain goods in a different way). The only thing we have to specify is the name of the module:

In [22]:
M.stdTrade('T');

### 1.6. Inventory investments

We include inventory investments for completeness, but simply keep them at an exogenous level for now. The default option uses the sector ``` s = 'itory'``` as the inventory investment sector:

In [23]:
M.stdInventory('IVT');

### 1.7. Index fund extension:

In [24]:
M.stdIndexFund('IdxF')

Welfare module:

In [25]:
M.stdWelfare('W', active = False) # don't use NLP solver yet.

## 2. Small stuff

*Clean up database a bit (this is not necessary, but it removes some variables that are not ultimately used in the model):*

In [26]:
[db.series.__delitem__(k) for k in ('vD','vTax', 'vD_dur','vD_depr','vD_inv', 'vS', 'pD_dur') if k in db.symbols];

For variables that are defined over $t$, but where we do not yet have an initial value for all $t$, extrapolate from data:

*Note: This forces extrapolation of all variables defined over $t$ - if it is important that some variables are not extrapolated, they should be removed from this statement.*

In [27]:
[symbol.__setattr__('vals', extrapolateUpper(symbol.vals, db('tE')[0])) for symbol in [db[k] for k in db.varDom('t')['t']]];

Merge internally, i.e. write a gdx file from the Python database:

In [28]:
db.mergeInternal()

## 3. Solve

*Note: when we initialized (some of) the modules, we added  the condition ```partial = True```. This statement allows us to run the modules separately. For instance, the following cell would solve the production module by itself and add the solution to the main database. This can be used as a way of getting good initial values for the model or to run partial equilibrium experiments.*

*Example for how to run a module separately and store the result in the main database:*

In [29]:
# m = M.m['P'] # select module to run 
# soldb = m.jSolve(10, state = 'C', ϕ = .1) # solve calibration model with 10 steps and nonlinear grid (ϕ<1 means that adjustments to jTerms start large and then decrease)
# [m.db.aom(soldb[k], name = k) for k in m.groups[f'{m.name}_endo_C'].out]; # for all endogenous variables, add the solution to the main database again 
# m.db.mergeInternal() # update the gdx with the new solution

Calibrate the model without equilibrium constraints yet (the jSolve method automatically uses all the modules that we have added so far):

In [30]:
soldb = M.jSolve(25, state = 'C', ϕ = .5) # solve calibration model with 10 steps and nonlinear grid (ϕ<1 means that adjustments to jTerms start large and then decrease)

*Note: GAMS automatically makes objects sparse (and does not store zeros). In this case, the tax rates are removed (as they are zero in this baseline):*

In [31]:
soldb['tauCO2agg'] = soldb('tauCO2agg').combine_first(M.db('tauCO2agg'))

Write solution to the main database again:

In [32]:
[M.db.__setitem__(k, soldb[k]) for k in M.db.getTypes(['var']) if k in soldb.symbols]; # use solution database
M.db.mergeInternal()

Remove "init" methods and set state to general equilibrium (not partial):

In [33]:
[m.__setattr__('initFromGms', None) for m in M.m.values() if hasattr(m, 'initFromGms')]; # remove the GAMS initialization part
[m.__setattr__('partial', False) for m in M.m.values() if hasattr(m, 'partial')]; # remove partial eq. settings for now also.

Now, add emissions and abatement module + equilibrium module. Calibrate in general equilibrium:

In [34]:
M.stdEquilibrium('Equi')
M.stdEmissions('M')
fullSol = M.jSolve(25, state = 'C')

Add full solution to baseline model database:

In [35]:
[M.db.__setitem__(k, fullSol[k]) for k in M.db.getTypes(['var']) if k in fullSol.symbols]; # use solution database
[m.__setattr__('initFromGms', None) for m in M.m.values() if hasattr(m, 'initFromGms')]; # remove the GAMS initialization part

## 4. Save/export

In [36]:
M.db.data_folder = d['data']

We can save/export the model in a few different ways:
1. Save model instance: ```Model``` (ultimate parent class for all the models) is pickleable, meaning that we can save/load the class with Python's ```pickle``` class.
2. Store solution: It takes no virtually no time to re-compile the ```Model``` class, so we can also simply store the solution database and - when needed - initialize the model class again.
3. Store GAMS code and solution: This option allows us to remove the model from the python class ```Model``` and instead treat it as a conventional GAMS program.

*1. Store model instance or database with pickle:*

In [37]:
M.export(repo = M.data_folder, name = M.name) # store the entire class - these are the default options by the way 

*2. Store data:*

In [38]:
# M.db.export(repo = M.db.data_folder, name = M.db.name) # store the solution database  - these are the default options by the way 

*3. Export gams code and accompanying gdx file:*

In [39]:
# baselineText = M.write(state = 'B') # store text so we can export it for later
# text_gamY    = M.write_gamY(state = 'B') # store gamY text

*Save text and accompanying gdx file:*

In [40]:
# with open(os.path.join(d['gams'], f'{M.name}.gms'), "w") as file:
#     file.write(baselineText)
# with open(os.path.join(d['gams'], f'{M.name}.gmy'), "w") as file:
#     file.write(text_gamY)
# M.db.database.export(os.path.join(M.db.data_folder, M.db.name))